In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

In [3]:
batch_size = 256
n_input = 784

In [6]:
def encoder(x):
    # 首先压缩成256，然后压缩成128
    weights1 = tf.Variable(tf.random_normal([784, 256]))
    biases1 = tf.Variable(tf.random_normal([256]))
    layer1 = tf.nn.sigmoid(tf.matmul(x, weights1) + biases1)
    # 第二层，压缩成128
    weights2 = tf.Variable(tf.random_normal([256, 128]))
    biases2 = tf.Variable(tf.random_normal([128]))
    layer2 = tf.nn.sigmoid(tf.matmul(layer1, weights2) + biases2)
    
    return layer2

def decoder(x):
    # 首先先从128还原成256，然后再从256还原会784
    weights1 = tf.Variable(tf.random_normal([128, 256]))
    biases1 = tf.Variable(tf.random_normal([256]))
    layer1 = tf.nn.sigmoid(tf.matmul(x, weights1) + biases1)
    # 从256还原成784
    weights2 = tf.Variable(tf.random_normal([256, 784]))
    biases2 = tf.Variable(tf.random_normal([784]))
    layer2 = tf.nn.sigmoid(tf.matmul(layer1, weights2) + biases2)
    
    return layer2

# 定义一个placeholder
X = tf.placeholder(tf.float32, [None, 784])

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
prediction = decoder_op

# 定义Optimizer
loss = tf.reduce_mean(tf.square(prediction - X))
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

# 初始化变量initial
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    total_batch = mnist.train.num_examples // batch_size
    
    # 开始训练
    for epoch in range(50):
        
        for i in range(total_batch):
            
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 训练并获得loss值
            _, c = sess.run([train_step, loss], feed_dict={X : batch_xs})
        
        if epoch % 5 == 0:
            print("第 %04d 个循环，loss是: %.9f" %(epoch + 1, c))
    
    encode_decode = sess.run(prediction, feed_dict={X : mnist.test.images[:10]})
    
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(10):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
    plt.show()

第 0001 个循环，loss是: 0.073095858
第 0006 个循环，loss是: 0.048142403
第 0011 个循环，loss是: 0.042466227
第 0016 个循环，loss是: 0.040883873
第 0021 个循环，loss是: 0.040777501
第 0026 个循环，loss是: 0.039862007
第 0031 个循环，loss是: 0.037560124
第 0036 个循环，loss是: 0.036317248
第 0041 个循环，loss是: 0.037167419
第 0046 个循环，loss是: 0.036537968
